In [ ]:
FILE_LIST = 'files.p'
DATA_DIR = "/run/media/maciej/Nowy/data/json/"
CHOSEN_YEAR = "2011"

In [ ]:
import os, json, pickle, sys, os, glob, time
from collections import defaultdict
from urllib.request import urlopen, Request

from tqdm import tqdm
import regex

from clarin import process


In [ ]:
def get_first_k_judgments(k=100):
    files = pickle.load(open(FILE_LIST, 'rb'))

    total_judgments = []
    for file in tqdm(files):
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [
                    (x["id"], x["judgmentDate"]) for x in data["items"] 
                    if x["judgmentDate"].startswith(CHOSEN_YEAR)
                ]
            total_judgments += judgments

    sorted_by_dates = sorted(total_judgments, key=lambda x: x[1])
    ids, _ = zip(*sorted_by_dates)
    
    return ids[:k]

first_100 = get_first_k_judgments()

In [ ]:
word_pattern = "\p{Letter}+"

def load_data(ids):
    total_judgments = []
    files = pickle.load(open(FILE_LIST, 'rb'))
    
    for file in files:
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [
                    x["textContent"] for x in data["items"] 
                    if x["id"] in ids
                ]
                
            total_judgments += judgments
            
    filtered_judgments = []
    for judgment in tqdm(total_judgments):
        judgment = regex.sub("<.*?>", "", judgment)
        judgment = regex.sub("-\n(\p{Letter}+)", r"\1", judgment)
        judgment = judgment.lower()
        
        filtered_judgments.append(judgment)
        
    
    return filtered_judgments

judgments = load_data(first_100)

In [ ]:
processed_judgments = []
for judgment in judgments:
    processed_judgment = process(judgment)
    if processed_judgment:
        processed_judgments.append(processed_judgment)
    else:
        print(processed_judgment)